In [1]:

data = []
date = []
title=[]
topic=[]
article=[]

import pandas as pd

data = pd.read_csv('/Users/melisince/Desktop/text analytics/Comp-Text-Analysis-2018-19-master/datasets/dataset.tsv',
                         sep='\t', header=0) #I downloaded the tsv file as a pandas dataframe and I informed python that the file is tab seperated by using sep='\t'
#so now it is going to be easy to extract the specific columns from the dataframe by their name.
data.columns=['date','title','topic','content'] #here I renamed the name of the columns because I was not sure if columns are called as they are written on the tsv file

data=data.iloc[1:] #above header=0 did not exclude the header row, so I tried this iloc. 

wordcount=data['content'].str.split().str.len() #the words are splitted by the white spaces, however the numbers and special characters like 
#twitter links or photo links for the articles are counted as a word here. 

data['wordcount']=wordcount #Here I created a new column called 'wordcount' and put the word counting of the content column into the wordcount column

data=pd.DataFrame(data[data.wordcount>6]) #dropped the articles which have wordcounts less than 6. from now on, contents with longer than 6 words are going to be in article corpus.

#here I assign each column to its so called list.
article=data.content.tolist() 
title=data.title.tolist()
date=data.date.tolist()
topic=data.topic.tolist()

print(len(article)) 

11132


Starting point:

Take your corpus and topic lists.

Train 1 type of topic classifier of your choice (among naive-bayes, knn and NearestCentroid) using pre-trained word embeddings to generate the feature-vector.

Task:

Compare the performance (always using 10 fold cross validation!) of at least 2 different pre-processing steps (for instance lemmatization vs non-lemmatization, stopword removal or not, only using NER instead of using lemmas, only using nouns, only using name of organizations).

Present here:

The two different text_embedding functions you used (example, a pipeline where you use lemmatization and one without lemmatization);
The cross-validation experiment, showing the different outputs in terms of precision, recall and F1

In [2]:

!pip install gensim #gensim is a library famous for word embeddings and semantics

import gensim, logging
model = gensim.models.KeyedVectors.load_word2vec_format('/Users/melisince/Desktop/text analytics/Comp-Text-Analysis-2018-19-master/resources/small-embeddings.txt', binary=False)

import nltk, string #nltk is for the tokenization and string is for extracting the punctuation.
from nltk.corpus import stopwords #I want to remove the stopwords which makes it easier for the function to clean the articles.. 
from nltk.stem.wordnet import WordNetLemmatizer #for the lemmatization. 
wordnet_lemmatizer = WordNetLemmatizer()

You are using pip version 18.1, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
exclude = set(string.punctuation) #list of punctuations to remove them
stop_word_list = stopwords.words('english') #extracting the list of stopwords in english.


def text_embedding_first(text):
    
    text = nltk.word_tokenize(text)#tokenization
    text = [token for token in text if token not in exclude and token.isalpha()]#removing characters that are not words in english and removing the punctuations. 
    text = [token for token in text if token not in stop_word_list] #keeping the words which are not in stop words lists
    text = nltk.pos_tag(text) #pos tagging for lemmatization 
    text = [wordnet_lemmatizer.lemmatize(token,"v") if pos[0] == "V" else wordnet_lemmatizer.lemmatize(token) for token,pos in text] #lemmatization 
    article_embedd = [] #wcreating an empty list for the word embeddings.
    #the for loop below will do the word embedding depending on the word2vec model
    for word in text: 
            try:
                embed_word = model[word] #model is the vocab of word embeddings where words are associated with word embeddings. 
                article_embedd.append(embed_word)
            except KeyError:
                continue
    avg = [float(sum(col))/len(col) for col in zip(*article_embedd)] #average vectors of all words in the articles. this is the column wise average. 
    return avg

In [4]:
#creating first corpus for the text embedding with lemmatization and removed stopwords. 
corpusfirst = []

for word in article:
    emb_text = text_embedding_first(word) #word embeddings are created. 
    if len(emb_text) > 0: #take the articles with word embeddings bigger than zero.
        corpusfirst.append(emb_text)
    
print("done!")

done!


In [5]:
print(len(corpusfirst))

11132


In [6]:
!pip install sklearn

import numpy as np


# I use np array as they are more efficient
X = np.array(corpusfirst)
y = np.array(topic)

from sklearn import cross_validation
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

final_f1first = [] 
final_precisionfirst=[]
final_recallfirst=[]

#we set that we do 10 fold cross validation

kf_total = cross_validation.StratifiedKFold(y, n_folds=10, shuffle=True)

for train, test in kf_total:
    X_train, X_test = X[train], X[test]
    y_train, y_test = y[train], y[test]
    
    classifier = GaussianNB().fit(X_train, y_train)
    
    y_pred = classifier.predict(X_test)
    
    print (precision_recall_fscore_support(y_test, y_pred, average="macro")) #if you are only interested in positive values, macro average between the two classes.
    f1_score = precision_recall_fscore_support(y_test, y_pred, average="macro")[2]
    precision_score = precision_recall_fscore_support(y_test, y_pred, average="macro")[0]
    recall_score = precision_recall_fscore_support(y_test, y_pred, average="macro")[1]
    final_f1first.append(f1_score)
    final_precisionfirst.append(precision_score)
    final_recallfirst.append(precision_score)
print (" ")
print (sum(final_f1first)/len(final_f1first))
print (sum(final_precisionfirst)/len(final_precisionfirst))
print (sum(final_recallfirst)/len(final_recallfirst))

You are using pip version 18.1, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:553: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


(0.47454354863390136, 0.536035178065942, 0.4823216023002924, None)
(0.5111812796680445, 0.5513811702834961, 0.5128340418878011, None)
(0.592252326817665, 0.6330349598758672, 0.5916261631234289, None)
(0.5426847050813826, 0.5688324115477217, 0.529202420834845, None)
(0.49014867867145523, 0.5208082740711054, 0.48132121039483966, None)
(0.52623601653576, 0.5552772617867748, 0.5148950420417192, None)
(0.5604681122454368, 0.655441114067207, 0.5698649734002443, None)
(0.5193255845493087, 0.5506725464111979, 0.5113872293898842, None)
(0.5270620003157697, 0.5555850660196245, 0.5146447809393302, None)
(0.42435940998586896, 0.44277215377981854, 0.4121870778921966, None)
 
0.5120284542204582
0.5168261662504592
0.5168261662504592


/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [7]:
#the second text preprocessing function without lemmatization
def text_embedding_second(text):
    
    #here I tokenized the each word for the preprocessign the articles. 
    text = nltk.word_tokenize(text)
    text = [token for token in text if token not in exclude and token.isalpha()]#removing characters that are not words in english and removing the punctuation basically. 
    text = [token for token in text if token not in stop_word_list] #keeping the words which are not in stop words lists
    article_embedd = []

    for word in text: 
            try:
                embed_word = model[word] #model is the vocab of word embeddings where words are associated with word embeddings. 
                article_embedd.append(embed_word)
            except KeyError:
                continue
    avg = [float(sum(col))/len(col) for col in zip(*article_embedd)] #average vectors of all words. this is the column wise average. this represent the meaning of your document.
    return avg

In [8]:
corpussecond = []

for word in article:
    emb_text = text_embedding_second(word) 
    if len(emb_text) > 0:
        corpussecond.append(emb_text)
    
print("done!")

done!


In [9]:
# I use np array as they are more efficient
X = np.array(corpussecond)
y = np.array(topic)

from sklearn import cross_validation
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

final_f1second = []
final_precisionsecond=[]
final_recallsecond=[]
#we set that we do 10 fold cross validation

kf_total = cross_validation.StratifiedKFold(y, n_folds=10, shuffle=True)

for train, test in kf_total:
    X_train, X_test = X[train], X[test]
    y_train, y_test = y[train], y[test]
    
    classifier = GaussianNB().fit(X_train, y_train)
    
    y_pred = classifier.predict(X_test)
    
    print (precision_recall_fscore_support(y_test, y_pred, average="macro")) #if you are only interested in positive values, macro average between the two classes.
    f1_score = precision_recall_fscore_support(y_test, y_pred, average="macro")[2]
    precision= precision_recall_fscore_support(y_test, y_pred, average="macro")[0]
    recall=precision_recall_fscore_support(y_test, y_pred, average="macro")[1]
    final_f1second.append(f1_score)
    final_precisionsecond.append(precision)
    final_recallsecond.append(recall)
print (" ")
print (sum(final_f1second)/len(final_f1second)) #the average of fscore
print(sum(final_precisionsecond)/len(final_precisionsecond)) #the average of precision
print(sum(final_recallsecond)/len(final_recallsecond))#the average of recall

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:553: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(0.5213164243418563, 0.5444049524159026, 0.5075616326822531, None)
(0.5489818638644993, 0.5682009340998503, 0.5363153263259005, None)
(0.5252300090145604, 0.5702324942201641, 0.5219464976888859, None)
(0.5304272106782718, 0.5703952992878674, 0.530456077759527, None)
(0.5341906146007918, 0.5657281412659783, 0.5259583690697713, None)
(0.5521343256751966, 0.6498443443027301, 0.5534561834865527, None)
(0.46615233836597686, 0.502412022694008, 0.46612346630351614, None)
(0.5436595607358959, 0.5786126948575565, 0.5354904633043461, None)
(0.5061562127161328, 0.5395705969387871, 0.5027989114308732, None)
(0.5564193093780203, 0.6287818205973289, 0.5597676889283619, None)
 
0.5239874616979988
0.5284667869371201
0.5718183300680173


/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [ ]:
#the precision, recall and f1 scores are changing between two texts which shows that text preprocessing matters. 
#Without lemmatization in the second text preprocessing, the precision, f1 and recall scores shows better performance. 

Take your corpus list.

Task:

Lemmatize the corpus, remove stopwords and lemmas shorter than 5 characters. Then convert each article in a document embedding.

Group documents in 10 clusters using k-means.

Counts how many documents you have, for each cluster.

In [10]:
wordnet_lemmatizer = WordNetLemmatizer()
stop_word_list = stopwords.words('english') 

def text_embedding_third(text):
    
    text = nltk.word_tokenize(text)#here I tokenized the each word for the preprocessign the articles.
    text = [token for token in text if token not in stop_word_list] #keeping the words which are not in stop words lists
    text = nltk.pos_tag(text)
    text = [wordnet_lemmatizer.lemmatize(token,"v") if pos[0] == "V" else wordnet_lemmatizer.lemmatize(token) for token,pos in text] 
    lemma_article=[] #here the list will contain lemma longer than 5 characters. 
    for lemma in text:
        if len(lemma)>5:
            lemma_article.append(lemma)
    article_embedd = [] #I want to use our vocabulary of word embeddings
# for each word in the article, I take the embeddings. I have a gigantic list of word embeddings of each word. I simply do a column wise average. then I have your doc embedding.
    for word in lemma_article: 
            try:
                embed_word = model[word] #model is the vocab of word embeddings where words are associated with word embeddings. 
                article_embedd.append(embed_word)
            except KeyError:
                continue
    avg = [float(sum(col))/len(col) for col in zip(*article_embedd)] #average vectors of all words. this is the column wise average. 
    return avg

corpusthird = []

for word in article:
    emb_text = text_embedding_third(word) #this function does all the things we have learnt throughout the lecture. in the exam, when you do text preprocessing write a function that takes your string into a vector.
    if len(emb_text) > 0:
        corpusthird.append(emb_text)
    
print("done!")

done!


In [11]:
print(len(corpusthird))

11121


In [12]:
from sklearn.cluster import KMeans #here is the library for KMeans. 
import numpy as np
k = 10 #setting the number of cluster
x = np.array(corpusthird)
model = KMeans(k, init='k-means++', max_iter=100, n_init=1).fit(x)

In [13]:
from collections import Counter,defaultdict #counting the documents in each cluster. 
print(Counter(model.labels_))

Counter({2: 1840, 3: 1803, 5: 1552, 8: 1483, 7: 966, 4: 947, 6: 914, 0: 850, 9: 485, 1: 281})


Take your corpus list.

Task:

Lemmatize the corpus and keep only the nouns.

Run LDA topic models with 50 topics.

Feed the results into pyLDAvis to visualize them

In [14]:
#the pipeline for the lemmatization of the nouns.
def text_lemma(text):
    tokentext = nltk.word_tokenize(text)
    
    cleantext = [token for token in tokentext if token not in exclude and token.isalpha()]

    pos_text = nltk.pos_tag(cleantext)
    
    nouns= [] #the list of the nouns.
    for word,pos in pos_text:
        if (pos=="NN" or pos == "NNP" or pos=="NNS" or pos=="NNPS"):
            nouns.append(word)
    lemmatext = [wordnet_lemmatizer.lemmatize(token) for token in nouns]
    
    return(lemmatext)

In [15]:
lemmano=[]#the final list that I will use in the LDA model dictionary
for word in article:
    lemmanouns=text_lemma(word)
    lemmano.append(lemmanouns)

In [16]:
print(lemmano[1])

['READ', 'MORE', 'plane', 'football', 'team', 'crash', 'Colombia', 'report', 'Ronaldinho', 'Argentina', 'star', 'Juan', 'Roman', 'Riquelme', 'club', 'Football', 'Ronaldinho', 'Juan', 'Roman', 'Riquelme', 'retirement', 'play', 'Chapecoensehttps', 'Ronaldinho', 'honor', 'playing', 'career', 'World', 'Cup', 'Riquelme', 'time', 'Argentina', 'country', 'player', 'year', 's', 'brother', 'agent', 'Roberto', 'Assis', 'Ronaldinho', 'club', 'way', 'discussion', 'place', 'time', 'Assis', 'Globo', 'Esporte', 'Later', 'contact', 'Ronaldinho', 'profile', 'guy', 'moment', 'family', 'expectation', 'Brazilians', 'plane', 'crash', 'people', 'club', 's', 'player', 'A', 'player', 'Danilo', 'hospital', 'wreckage', 'FIFA', 'president', 'Gianni', 'Infantino', 'funeral', 'Chapecoense', 'player', 'Friday', 'trip', 'Australia', 'woman', 'World', 'Cup', 'crash', 'world', 'UEFA', 'president', 'Aleksander', 'Ceferin', 'condolence', 'week', 'Champions', 'League', 'Europa', 'League', 'game', 's', 'silence', 'player'

In [17]:
from gensim import corpora, models #the packages are downloaded

In [18]:
#For each article in the corpus, creating a dictionary 
dictionary = corpora.Dictionary(lemmano) 
A = [dictionary.doc2bow(text) for text in lemmano]
print("ready!")

ready!


In [19]:
ldamodel = models.ldamodel.LdaModel(A, num_topics=50, id2word = dictionary, iterations=1000) 
print ("done!")

done!


In [20]:
for topic in ldamodel.print_topics(num_words=30):
    print (topic)

(9, '0.030*"photo" + 0.023*"A" + 0.020*"PDT" + 0.018*"GIPHY" + 0.011*"bear" + 0.011*"project" + 0.009*"PST" + 0.009*"technology" + 0.007*"year" + 0.007*"park" + 0.007*"Oct" + 0.007*"Instagram" + 0.005*"program" + 0.005*"world" + 0.005*"s" + 0.005*"design" + 0.005*"cloud" + 0.004*"brain" + 0.004*"team" + 0.004*"animal" + 0.004*"time" + 0.004*"visitor" + 0.004*"Nov" + 0.003*"Park" + 0.003*"vehicle" + 0.003*"Technology" + 0.003*"manufacturer" + 0.003*"efficiency" + 0.003*"Algiers" + 0.003*"engineer"')
(10, '0.040*"UK" + 0.036*"EU" + 0.034*"Britain" + 0.021*"Brexit" + 0.016*"referendum" + 0.015*"Cameron" + 0.010*"vote" + 0.009*"s" + 0.009*"people" + 0.008*"Minister" + 0.008*"country" + 0.008*"minister" + 0.008*"leader" + 0.008*"Prime" + 0.008*"David" + 0.007*"London" + 0.007*"Europe" + 0.007*"member" + 0.006*"European" + 0.006*"PM" + 0.006*"government" + 0.006*"year" + 0.006*"May" + 0.005*"Johnson" + 0.005*"campaign" + 0.005*"time" + 0.005*"Parliament" + 0.005*"MP" + 0.005*"decision" + 0.0

In [ ]:
#I have tried to import the package but it did not work in python 3. So i leave my codes here to show I have tried and I got an error.. 

## Interactive visualisation
!pip forge pyldavis install#it does not function.
import pyLDAvis.gensim
vis = pyLDAvis.gensim.prepare(ldamodel, lemmano, dictionary)
pyLDAvis.display(vis)